In [1]:
import pymongo
import pandas as pd

In [14]:
MongoClient = pymongo.MongoClient
client = MongoClient()
db = client.companies

In [19]:
def toGeoJSON(lista):
    for i in lista:
        return {
            "type":"Point",
            "coordinates":(i['longitude'],i['latitude'])
        }

def convertRestaurant(company):
    for diccio in company["offices"]:
            
    return {
        "name": company["name"],
        "homepage_url": company["homepage_url"],
        "twitter_username": company["twitter_username"],
        "category_code": company["category_code"],
        "number_of_employees": company["number_of_employees"],
        "deadpooled_year" : company["deadpooled_year"],
        "founded_year": company["founded_year"],     
        "funding_rounds": company["funding_rounds"],
        "position": toGeoJSON(company["offices"])
    }

cursor = db.companies.find().limit(5)
geoCompanies = list(map(lambda r: convertRestaurant(r),cursor))
geoCompanies

[{'name': 'AdventNet',
  'homepage_url': 'http://adventnet.com',
  'twitter_username': 'manageengine',
  'category_code': 'enterprise',
  'number_of_employees': 600,
  'deadpooled_year': 2,
  'founded_year': 1996,
  'funding_rounds': [],
  'position': {'type': 'Point', 'coordinates': (-121.904945, 37.692934)}},
 {'name': 'Wetpaint',
  'homepage_url': 'http://wetpaint-inc.com',
  'twitter_username': 'BachelrWetpaint',
  'category_code': 'web',
  'number_of_employees': 47,
  'deadpooled_year': 1,
  'founded_year': 2005,
  'funding_rounds': [{'id': 888,
    'round_code': 'a',
    'source_url': 'http://seattlepi.nwsource.com/business/246734_wiki02.html',
    'source_description': '',
    'raised_amount': 5250000,
    'raised_currency_code': 'USD',
    'funded_year': 2005,
    'funded_month': 10,
    'funded_day': 1,
    'investments': [{'company': None,
      'financial_org': {'name': 'Frazier Technology Ventures',
       'permalink': 'frazier-technology-ventures'},
      'person': None},


In [21]:
df = pd.DataFrame(geoCompanies)
df.head()

,category_code,deadpooled_year,founded_year,funding_rounds,homepage_url,name,number_of_employees,twitter_username
0,enterprise,2.0,1996,[],http://adventnet.com,AdventNet,600.0,manageengine
1,web,1.0,2005,"[{'id': 888, 'round_code': 'a', 'source_url': ...",http://wetpaint-inc.com,Wetpaint,47.0,BachelrWetpaint
2,software,3.0,2005,[],http://zoho.com,Zoho,1600.0,zoho
3,news,NaN,2004,"[{'id': 1, 'round_code': 'b', 'source_url': 'h...",http://www.digg.com,Digg,60.0,digg
4,network_hosting,2008.0,2005,"[{'id': 225, 'round_code': 'angel', 'source_ur...",http://www.omnidrive.com,Omnidrive,NaN,Nomadesk


In [ ]:
category_code
deadpooled_year
founded_year
funding_rounds     raised_amount
number_of_employees
offices      latitude
            longitude
    
    "offices": 
        deadpoolyear: None